In [12]:
import pandas as pd
import numpy as np

In [2]:
test_df = pd.read_csv("test_table.csv")
user_df = pd.read_csv("user_table.csv")

In [3]:
test_df.head()

,user_id,date,source,device,browser_language,ads_channel,browser,conversion,test
0,315281,2015-12-03,Direct,Web,ES,NaN,IE,1,0
1,497851,2015-12-04,Ads,Web,ES,Google,IE,0,1
2,848402,2015-12-04,Ads,Web,ES,Facebook,Chrome,0,0
3,290051,2015-12-03,Ads,Mobile,Other,Facebook,Android_App,0,1
4,548435,2015-11-30,Ads,Web,ES,Google,FireFox,0,1


In [7]:
test_df.describe()

,user_id,conversion,test
count,453321.000000,453321.000000,453321.000000
mean,499937.514728,0.049579,0.476446
std,288665.193436,0.217073,0.499445
min,1.000000,0.000000,0.000000
25%,249816.000000,0.000000,0.000000
50%,500019.000000,0.000000,0.000000
75%,749522.000000,0.000000,1.000000
max,1000000.000000,1.000000,1.000000


In [4]:
user_df.head()

,user_id,sex,age,country
0,765821,M,20,Mexico
1,343561,F,27,Nicaragua
2,118744,M,23,Colombia
3,987753,F,27,Venezuela
4,554597,F,20,Spain


In [8]:
user_df.describe()

,user_id,age
count,452867.000000,452867.000000
mean,499944.805166,27.130740
std,288676.264784,6.776678
min,1.000000,18.000000
25%,249819.000000,22.000000
50%,500019.000000,26.000000
75%,749543.000000,31.000000
max,1000000.000000,70.000000


In [6]:
print("Unique users in users: {}".format(len(user_df.user_id.unique())))
print("Unique users in test: {}".format(len(test_df.user_id.unique())))

Unique users in users: 452867
Unique users in test: 453321


In [9]:
df = pd.merge(test_df, user_df, how="left")

In [18]:
for column in df.columns:
    print("Column: " + column)
    print(df[column].describe())
    print()

Column: user_id
count     453321.000000
mean      499937.514728
std       288665.193436
min            1.000000
25%       249816.000000
50%       500019.000000
75%       749522.000000
max      1000000.000000
Name: user_id, dtype: float64

Column: date
count         453321
unique             5
top       2015-12-04
freq          141163
Name: date, dtype: object

Column: source
count     453321
unique         3
top          Ads
freq      181877
Name: source, dtype: object

Column: device
count     453321
unique         2
top          Web
freq      251565
Name: device, dtype: object

Column: browser_language
count     453321
unique         3
top           ES
freq      377547
Name: browser_language, dtype: object

Column: ads_channel
count       181877
unique           5
top       Facebook
freq         68425
Name: ads_channel, dtype: object

Column: browser
count          453321
unique              7
top       Android_App
freq           155135
Name: browser, dtype: object

Column: conversio

/home/sayan/anaconda3/lib/python3.5/site-packages/numpy/lib/function_base.py:3834: RuntimeWarning: Invalid value encountered in percentile
  RuntimeWarning)


In [69]:
df_sum = df.groupby(["country","test"], as_index=False)["conversion"].sum().rename(columns={'conversion':'total_converts'})
df_count = df.groupby(["country","test"], as_index=False)["conversion"].count().rename(columns={'conversion':'total_tested'})
agg_df = pd.merge(df_sum, df_count)
agg_df["conversion_rate"] = agg_df["total_converts"]*100/agg_df["total_tested"]
# df.groupby(["country","test"], as_index=False)["conversion"].count()
agg_df.head()

,country,test,total_converts,total_tested,conversion_rate
0,Argentina,0,141,9356,1.507054
1,Argentina,1,513,37377,1.372502
2,Bolivia,0,274,5550,4.936937
3,Bolivia,1,267,5574,4.790097
4,Chile,0,474,9853,4.810718


In [70]:
agg_df = pd.merge(agg_df, agg_df.groupby("country", as_index=False)["total_tested"].sum().rename(columns={'total_tested':'total_users'}))
agg_df["test_rate"] = agg_df["total_tested"]*100/agg_df["total_users"]
agg_df.sort_values(by="conversion_rate")

,country,test,total_converts,total_tested,conversion_rate,total_users,test_rate
29,Uruguay,0,5,415,1.204819,4134,10.038703
30,Uruguay,1,48,3719,1.290670,4134,89.961297
1,Argentina,1,513,37377,1.372502,46733,79.979886
0,Argentina,0,141,9356,1.507054,46733,20.020114
22,Panama,0,92,1966,4.679552,3951,49.759555
17,Honduras,1,200,4207,4.753981,8568,49.101307
3,Bolivia,1,267,5574,4.790097,11124,50.107875
13,El Salvador,1,195,4067,4.794689,8175,49.749235
4,Chile,0,474,9853,4.810718,19737,49.921467
24,Paraguay,0,177,3650,4.849315,7347,49.680142
